# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [19]:
import pandas as pd
import numpy as np
import pickle


import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [21]:
# Cache stop_words into hash
stop_words = stopwords.words('english')
stop_words.extend(['from'])
stop_words = Counter(stop_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

random.seed(3)

In [4]:
# nltk.download('stopwords')

## Preprocessing data

In [5]:
start = time.time()

In [6]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [7]:
df = pd.read_pickle('reviews_concat.pkl')

In [8]:
# df = df.sample(100, random_state=3)
# del df

In [9]:
# Cache stop_words into hash
stop_words = stopwords.words('english')
stop_words.extend(['from'])
stop_words = Counter(stop_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # lower case
    # Remove stop words
    # Lemmatize
    for sentence in sentences:
        doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True) if token not in stop_words]) )
        yield([token.lemma_ for token in doc if token.pos_ in allowed_postags])

In [10]:
docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

5131380it [3:00:24, 474.04it/s]


In [18]:
len(docs)

5131380

## Validating Model

In [61]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [24]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [33]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)*len(corpus_title)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()







  0%|                                                                                            | 0/2 [00:00<?, ?it/s]





 50%|█████████████████████████████████████████▌                                         | 1/2 [03:52<03:52, 232.60s/it]





100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [07:55<00:00, 237.63s/it]


In [34]:
res

,Validation_Set,Topics,Alpha,Beta,Coherence
0,100% Corpus,8,6.25,0.1,0.533193
1,100% Corpus,8,6.25,symmetric,0.531085


## Training Best Model

In [35]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [36]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

214.14255166053772


In [37]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.522543142845403


In [38]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.116661  0.060797       1        1  12.830737
7     -0.172795 -0.044542       2        1  12.794074
5      0.027699  0.282577       3        1  12.596339
6      0.267409 -0.052143       4        1  12.523667
2     -0.173926 -0.087534       5        1  12.435490
0      0.025170 -0.027751       6        1  12.384436
3      0.105066 -0.059904       7        1  12.253001
1      0.038037 -0.071500       8        1  12.182255, topic_info=      Term          Freq         Total Category  logprob  loglift
172  color  26952.000000  26952.000000  Default  30.0000  30.0000
35    skin  32937.000000  32937.000000  Default  29.0000  29.0000
187    lip  16217.000000  16217.000000  Default  28.0000  28.0000
144    eye  11569.000000  11569.000000  Default  27.0000  27.0000
472   nice  10958.000000  10958.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
248    way   1496.740234   5300.721680   Topic8  -4.9616   0.8406
24     use   2081.195068  39338.324219   Topic8  -4.6319  -0.8341
89    much   1600.839722   8224.726562   Topic8  -4.8943   0.4686
43     get   1656.439087  12468.770508   Topic8  -4.8602   0.0866
3      buy   1607.338013  12371.419922   Topic8  -4.8903   0.0644

[604 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
301       1  0.055299  absolutely
301       2  0.065353  absolutely
301       3  0.015673  absolutely
301       4  0.130411  absolutely
301       5  0.026319  absolutely
...     ...       ...         ...
354       8  0.048488        year
989       1  0.009988      yellow
989       6  0.986328      yellow
989       8  0.002497      yellow
723       1  0.998132        zone

[1819 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 6, 7, 3, 1, 4, 2])

In [39]:
lda_model.save('lda_test.model')

In [40]:
print(time.time()-start)

31247.86803984642


## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
docs = pickle.load(open( "docs.p", "rb" ))
id2word = pickle.load(open( "id2word.p", "rb" ))

In [ ]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [15]:
# lda_model = gen?sim.models.LdaModel.load('lda.model')

In [69]:
output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

In [ ]:
pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])

In [71]:
df.to_frame('description').reset_index()

,date,onlinepost_id,description
0,2014-12-01,OnlinePost_20191027_181650262,It's lightweight yet hides my imperfections.. ...
1,2014-12-01,OnlinePost_20191027_181666571,"Even with oily skin, I prefer the look of this..."
2,2014-12-01,OnlinePost_20191027_181666940,I wear foundation nearly every day & this is p...
3,2014-12-01,OnlinePost_20191027_181694251,i'm a mascara junky…. i really like this masca...
4,2014-12-01,OnlinePost_20191027_181695799,I received this as a sample in my GWP and love...
...,...,...,...
5131375,2020-04-30,OnlinePost_20200524_ff7e18ee-3f03-4bfe-bf9a-98...,"- expensive cuz its just a moisturizer, could ..."
5131376,2020-04-30,OnlinePost_20200524_ff907024-361c-4bc4-959e-0b...,I appreciate that this cleanser is very moistu...
5131377,2020-04-30,OnlinePost_20200524_ffadec77-5361-4a6b-9134-83...,It is sticky once applied but this doesn't bot...
5131378,2020-04-30,OnlinePost_20200524_ffcde3c4-2f75-4431-ba64-2e...,The pump isnt excessive either so you definite...


In [ ]:
output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)